1. **User Engagement:**
 - SELECT COUNT(UserID) AS NumberOfUsers FROM Users;
 - SELECT AVG(RegistrationDate) AS AvgRegistrationDate FROM Users;
 - SELECT UserID, RegistrationDate
   FROM Users
   WHERE RegistrationDate <= DATE_SUB(CURDATE(), INTERVAL 1 YEAR);


2. **Post Analysis:**
 - SELECT COUNT(PostID) AS NumberOfPosts
    FROM Posts;
 
 - SELECT AVG(LENGTH(PostText) - LENGTH(REPLACE(PostText, ' ', '')) + 1) AS AvgWordsPerPost
    FROM Posts;

 - SELECT UserID, AVG(LENGTH(PostText) - LENGTH(REPLACE(PostText, ' ', '')) + 1) AS AvgPostLength
    FROM Posts
    GROUP BY UserID
    ORDER BY AvgPostLength DESC
    LIMIT 3;

3. **Comment Analysis:**

 - SELECT COUNT(CommentID) AS NumberOfComments
    FROM Comments;
    
 - SELECT AVG(LENGTH(CommentText) - LENGTH(REPLACE(CommentText, ' ', '')) + 1) AS AvgWordsPerComment
    FROM Comments;
    
 - SELECT UserID, AVG(LENGTH(CommentText) - LENGTH(REPLACE(CommentText, ' ', '')) + 1) AS AvgCommentLength
    FROM Comments
    GROUP BY UserID
    ORDER BY AvgCommentLength DESC
    LIMIT 3;


4. **Like Analysis:**

  - SELECT COUNT(LikeID) AS NumberOfLikes FROM Likes;
  - SELECT AVG(LikeCount) AS AvgLikesPerUser FROM (
    SELECT UserID, COUNT(LikeID) AS LikeCount
    FROM Likes
    GROUP BY UserID
    ) AS UserLikeCounts;

  - SELECT UserID, AVG(LikeCount) AS AvgLikesReceived
    FROM (
    SELECT UserID, COUNT(LikeID) AS LikeCount
    FROM Likes
    GROUP BY UserID
    ) AS UserLikeCounts
    GROUP BY UserID
    ORDER BY AvgLikesReceived DESC
    LIMIT 3;

5. **User Interaction:**
  - SELECT UserID
    FROM (
    SELECT UserID FROM Posts
    INTERSECT
    SELECT UserID FROM Comments
    INTERSECT
    SELECT UserID FROM Likes
    ) AS UsersWithActivity;

  - SELECT PostID
    FROM (
    SELECT PostID FROM Comments
    INTERSECT
    SELECT PostID FROM Likes
    ) AS PostsWithCommentsAndLikes;
  - SELECT UserID, COUNT(PostID) AS NumPostsInDay
    FROM Posts
    GROUP BY UserID, DATE(PostDate)
    HAVING COUNT(PostID) > 1;
6. **Temporal Analysis:**
 - SELECT AVG(TimeGap) AS AvgTimeGap
    FROM (
    SELECT P.PostID, MIN(C.CommentDate - P.PostDate) AS TimeGap
    FROM Posts P
    LEFT JOIN Comments C ON P.PostID = C.PostID
    GROUP BY P.PostID
    ) AS TimeGaps;
 - SELECT WEEKDAY(PostDate) AS DayOfWeek, COUNT(PostID) AS NumPosts
    FROM Posts
    GROUP BY DayOfWeek
    ORDER BY NumPosts DESC
    LIMIT 1;
7. **Engagement Trends:**

 - SELECT
    DATE_FORMAT(PostDate, '%Y-%m') AS YearMonth,
    COUNT(PostID) AS NumPosts
    FROM Posts
    GROUP BY YearMonth
    ORDER BY YearMonth;

8. **Influence Analysis:**
 - WITH UserEngagement AS (
    SELECT UserID, COUNT(PostID) + COUNT(CommentID) + COUNT(LikeID) AS CumulativeEngagement
    FROM (
        SELECT UserID, PostID, NULL AS CommentID, NULL AS LikeID FROM Posts
        UNION ALL
        SELECT UserID, NULL AS PostID, CommentID, NULL AS LikeID FROM Comments
        UNION ALL
        SELECT UserID, NULL AS PostID, NULL AS CommentID, LikeID FROM Likes
    ) AS UserActivity
    GROUP BY UserID
    )
    SELECT UserID, CumulativeEngagement
    FROM UserEngagement
    ORDER BY CumulativeEngagement DESC
    LIMIT 3;
    
  - WITH PostEngagement AS (
    SELECT
        P.PostID,
        COUNT(C.CommentID) AS NumComments,
        COUNT(L.LikeID) AS NumLikes
    FROM Posts P
    LEFT JOIN Comments C ON P.PostID = C.PostID
    LEFT JOIN Likes L ON P.PostID = L.PostID
    GROUP BY P.PostID
    )
    SELECT PostID, NumComments, NumLikes, (NumComments + NumLikes) AS TotalEngagement
    FROM PostEngagement
    ORDER BY TotalEngagement DESC
    LIMIT 1;

9. **User Activity Patterns:**
  - SELECT
    UserID,
    AVG(PostsPerMonth) AS AvgPostsPerMonth
    FROM (
    SELECT
        UserID,
        DATE_FORMAT(PostDate, '%Y-%m') AS YearMonth,
        COUNT(PostID) AS PostsPerMonth
    FROM Posts
    GROUP BY UserID, YearMonth
    ) AS UserPostsPerMonth
    GROUP BY UserID;

10. **Post Engagement Analysis:**

 - SELECT
    AVG(NumComments) AS AvgCommentsPerPost,
    AVG(NumLikes) AS AvgLikesPerPost
    FROM (
    SELECT
        P.PostID,
        COUNT(DISTINCT C.CommentID) AS NumComments,
        COUNT(DISTINCT L.LikeID) AS NumLikes
    FROM Posts P
    LEFT JOIN Comments C ON P.PostID = C.PostID
    LEFT JOIN Likes L ON P.PostID = L.PostID
    GROUP BY P.PostID
    ) AS PostEngagement;

 - SELECT
    PostID,
    NumComments,
    NumLikes,
    CAST(NumComments AS DECIMAL) / NULLIF(NumLikes, 0) AS CommentsToLikesRatio
    FROM (
    SELECT
        P.PostID,
        COUNT(C.CommentID) AS NumComments,
        COUNT(L.LikeID) AS NumLikes
    FROM Posts P
    LEFT JOIN Comments C ON P.PostID = C.PostID
    LEFT JOIN Likes L ON P.PostID = L.PostID
    GROUP BY P.PostID
    ) AS PostEngagement
    ORDER BY CommentsToLikesRatio DESC
    LIMIT 3;
11. **Comment Length Distribution:**

 - SELECT
    CommentID,
    CommentText,
    LENGTH(CommentText) AS CommentLength
    FROM Comments
    ORDER BY CommentLength DESC;
 - SELECT
    UserID,
    AVG(LENGTH(CommentText)) AS AvgCommentLength
    FROM Comments
    GROUP BY UserID;
